# Solution Report

This file aims to study each approach, detailing it's main strategy as well as weighting in on it's advantages and disadvantages

**Important Note:** If you want to run this report, be sure you are using a **Linux** distribution

### Index
 - [Setting up the Environment](#setting-up-the-environment)
 - [Brute-Force](#brute-force)

## Setting up the Environment

To generate the example graphs, as well as checking if the cliques we found are correct, the `networkx` library is used. Let's make sure that this library is installed

In [8]:
%pip install networkx

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Note that there aren't any graphs in the directory currently. Let's generate them using the `generate_graph.py` in the `scripts` folder. We will generate just one graph for now to be used as a test for the algorithms.

In [17]:
# generate_graph.py args: argv[1] = n_edges, argv[2] = probability of connection, argv[3] = filename
%run scripts/generate_graph.py 500 0.3 test_graph.txt

Graph generated and saved at /home/gabrielhso/Desktop/insper/2023.2/superComp/maximum-clique-detection/graphs'.


## Brute-Force

### Description
The brute force approach to finding the maximum clique in a graph is a method that systematically explores all possible combinations of vertices to identify the largest complete subgraph, known as a clique. It starts with an initial vertex and iteratively adds vertices to a candidate clique, checking if the addition maintains the clique property. This exhaustive approach guarantees the optimality of the solution, but its computational complexity grows exponentially with the number of vertices, making it impractical for large graphs.

### Running the Code

As we will evaluate the efficiency of the algorithim itself, we won't compile it with the optimization options of the compiler (-O flag). Also, we will use the -g to include debug symbols.


In [18]:
# Compiling
!g++ -Wall -O3 -g scripts/brute_force.cpp -o ./scripts/brute_force

In [19]:
# Running
!./scripts/brute_force graphs/test_graph.txt

Size of the maximum clique: 9
Maximum clique found:
[17, 33, 53, 211, 237, 251, 313, 332, 380]


Let's check the answer using the `check_answer.py` file

In [20]:
%run scripts/check_answer.py graphs/test_graph.txt

Size of the maximum clique:  9
Maximum cliques found:
[17, 33, 53, 211, 237, 251, 313, 332, 380]


### Profiling

Now let's check the performance of this code

In [21]:
!valgrind --tool=callgrind ./scripts/brute_force graphs/test_graph.txt

==5994== Callgrind, a call-graph generating cache profiler
==5994== Copyright (C) 2002-2017, and GNU GPL'd, by Josef Weidendorfer et al.
==5994== Using Valgrind-3.18.1 and LibVEX; rerun with -h for copyright info
==5994== Command: ./scripts/brute_force graphs/test_graph.txt
==5994== 
==5994== For interactive control, run 'callgrind_control -h'.
Size of the maximum clique: 9
Maximum clique found:
[17, 33, 53, 211, 237, 251, 313, 332, 380]
==5994== 
==5994== Events    : Ir
==5994== Collected : 748994908
==5994== 
==5994== I   refs:      748,994,908


In [24]:
!callgrind_annotate callgrind.out.5994 scripts/brute_force.cpp --auto=no

--------------------------------------------------------------------------------
Profile data file 'callgrind.out.5994' (creator: callgrind-3.18.1)
--------------------------------------------------------------------------------
I1 cache: 
D1 cache: 
LL cache: 
Timerange: Basic block 0 - 106349944
Trigger: Program termination
Profiled target:  ./scripts/brute_force graphs/test_graph.txt (PID 5994, part 1)
Events recorded:  Ir
Events shown:     Ir
Event sort order: Ir
Thresholds:       99
Include dirs:     
User annotated:   scripts/brute_force.cpp
Auto-annotation:  off

--------------------------------------------------------------------------------
Ir                   
--------------------------------------------------------------------------------
748,994,908 (100.0%)  PROGRAM TOTALS

--------------------------------------------------------------------------------
Ir                    file:function
--------------------------------------------------------------------------------
570

Ao expandir a saída do `annotate`, é possível enxergar que as partes do código com maior proporção de execuções destinadas são:

Scripts e biblioteca
 - 76.21%: `brute_force.cpp` (o próprio código em c)
 - 12.04%: `stl_vector.h:main` (biblioteca vector)
 - 11.75%: outras bibliotecas

Não temos uma maneira de melhorar o código da biblioteca, então teremos que nos ater à mudança no nosso código.

Dentre as partes do programa, nota-se que: 
 - 4.83% das execuções são na função de ler o grafo `ReadGraph`
 - 75,83% das execuções ocorrem na seguinte parte:

```c++
( 0.03%)           for (int connection1 : connections)
                   {
                       vector<int> clique = {currentNode, connection1}; // Move initialization here
( 7.43%)               for (int connection2 : connections)
                       {
( 1.48%)                   bool inClique = true;
(24.16%)                   for (int member : clique)
                           {
(15.61%)                       if (graph[member][connection2] == 0)
(24.16%)                       inClique = false;
                           }
( 2.97%)                    if (inClique)
                                clique.push_back(connection2);
                        }
( 0.02%)                if (clique.size() > maximumClique.size())
                        maximumClique = clique;
                   }
               }
```

Esse código resolve o problema, mas devido ao alto número de dependências é possível 